In [4]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz

In [17]:
#Import Labcorp and SLAB from Excel. NOTE: 'new_slab_data' MUST ONLY CONTAINS ACCESSIONS THAT HAVE NOT ALREADY BEEN MATCHED
LCA_DATA = pd.read_excel("*******",sheet_name='Sheet2',usecols=[0,1,2,3,4,7])#LCA data will never change, it will always be this static file
old_slab_data = pd.read_excel(r"*******",sheet_name='LCA SLAB',usecols=[11,12,13,14,15,16,17])#old data should read the slab data from the FINAL export file.
new_slab_data = pd.read_excel(r"*******",sheet_name='NEW acc',usecols=[0,1,2,3,4,5,6])#whichever file has the most up-to-date SLAB accessions

In [18]:
#create dictionary of unique names in 'new_slab_data's "Name dob" column
nameDOB_unique = new_slab_data["Name dob"]
nameDOB_unique.drop_duplicates(keep="first", inplace=True)
fuzz_dict = dict.fromkeys(nameDOB_unique)

1142


In [26]:
%%time
threshold = 90
for name in fuzz_dict.copy():
    name_fuzz = process.extract(name, LCA_DATA['Name dob'], scorer = fuzz.ratio)
    for tup in name_fuzz.copy():
        if tup[1] < threshold:
            index = name_fuzz.index(tup)
            name_fuzz.pop(index)
    fuzz_dict[name] = name_fuzz

CPU times: total: 2.39 s
Wall time: 4.3 s


In [23]:
%%time

#Extract all accession information and put into a dataframe
def append_row(list, num_of_appends, backup_appends, dataframe): 
    for n in range(num_of_appends):
        list.append(dataframe)
        
    return list


matched_lca_list = []
matched_slab_list = []
test = 0

for name in fuzz_dict:
    if len(fuzz_dict[name]) != 0: #this prevents accessions with NO LCA match from being added to the list
        lca_row = LCA_DATA.loc[LCA_DATA['Name dob'] == name] #.loc returns ALL appearnces of 'Name dob'
        lca_occur = len(lca_row)
        
        slab_row = new_slab_data.loc[new_slab_data['Name dob'] == name]
        slab_occur = len(slab_row)

        matched_lca_list = append_row(matched_lca_list, slab_occur, lca_occur, lca_row)
        matched_slab_list = append_row(matched_slab_list, lca_occur, slab_occur, slab_row)
        
        #test += 1

lca_exportDF = pd.concat(matched_lca_list)
slab_exportDF = pd.concat(matched_slab_list)

In [24]:
#exporting DFs to Excel
# lca_append_col = #LCA COLUMNS IN EXPORT SHEET
# slab_append_col = #SLAB COLUMNS IN EXPORT SHEET
# append_row = #LAST ROW IN EXPORT SHEET

def excel_print(file, lca_data, slab_data, row_append, new):
    writer = pd.ExcelWriter(file, mode="a", if_sheet_exists="overlay")
    
    if new == 1:
        lca_data.to_excel(writer, sheet_name="LCA SLAB", startrow=0, startcol=0, index=False)
        slab_data.to_excel(writer, sheet_name="LCA SLAB", startrow=0, startcol=11, index=False, freeze_panes=[1,1])
        
    else:
        lca_data.to_excel(writer, sheet_name="LCA SLAB", startrow=row_append, startcol=0, index=False, header=False)
        slab_data.to_excel(writer, sheet_name="LCA SLAB", startrow=row_append, startcol=11, index=False, header=False, freeze_panes=[1,1])
    
    writer.close()

row_append = old_slab_data["Name dob.1"].index[-1]
export_file = r"*******"
excel_print(export_file, lca_exportDF, slab_exportDF, row_append, 0)# CHANGE LAST VARIABLE TO EITHER '0' OR '1' IF YOU WANT TO APPEND TO START CLEAN SHEET RESPECTIVELY
